# Tier 3, Week 7: The Variational Quantum Eigensolver (VQE)

**Audience:** Undergraduate / Advanced

**Objective:** Generalize the hybrid loop concept from Week 6 to solve physical problems. Understand how to map fermions (electrons) to qubits using the Jordan-Wigner mapping and use the VQE algorithm to calculate the ground state energy of molecules.

## 1. Bridging from Week 6: From QAOA to VQE

In Week 6, we explored **QAOA** (Quantum Approximate Optimization Algorithm) to solve the **Max-Cut** problem. The **Goal** is to find a bitstring $x$ that minimizes a cost function $C(x)$. To achieve it, we constructed a cost Hamiltonian $H_C$ such that its ground state (lowest energy state) encoded the solution. We used a specific ansatz (alternating layers of Cost and Mixer unitaries) to find this ground state.

**Week 7: The Generalization**

QAOA is actually just a specific instance of a broader class of algorithms called **VQE (Variational Quantum Eigensolver)**.

* **In QAOA:** We *invented* the Hamiltonian $H_C$ to represent a man-made graph problem.

* **In VQE:** Nature gives us the Hamiltonian $H$. It represents the energy of a physical system, like a molecule. From nature, we find the **Ground State Energy** of that molecule. This is the most stable configuration of the system and determines reaction rates, binding energies, and molecular geometry.

The core loop remains the same: **Quantum Computer** prepares a state $\rightarrow$ **Estimator** measures energy $\rightarrow$ **Classical Computer** updates parameters.



## 2. The Physics Problem: Electronic Structure

 ### What is the Schrödinger Equation?
 
 At the heart of quantum chemistry lies the Schrödinger Equation. The Schrödinger Equation describes the wave function of the particle. While you might seen it describing how particles move over time (in the time-dependent form), for VQE we care about the Time-Independent Schrödinger Equation:
 $$
 H |\psi\rangle = E |\psi\rangle
 $$
 The equation above is the form after factoring out the time-dependent variable. This is an Eigenvalue Problem. It asks: "Which states $|\psi\rangle$ remain unchanged (pointing in the same direction) when the operator $H$ is applied to them?"
 
 Here is the breakdown of the components:
 * $H$ (The Hamiltonian):  This is the operator representing the Total Energy of the system. For a molecule, it sums up all the interactions: 
    * The *Kinetic energy* of the electrons.
    * Attraction between negatively charged electrons and positively charged nuclei.
    * Repulsion between electrons (this is the hard part to simulate!).
    
* $|\psi\rangle$ (The Eigenstate / Wavefunction): This vector represents a stable configuration of the electrons. If the system is in this state, its energy is definite and constant.

* $E$ (The Eigenvalue): The specific scalar value representing the energy of the state $|\psi\rangle$. 

A system naturally wants to "relax" to its lowest possible energy. Therefore, our goal is to find the Ground State, the eigenvector $|\psi_0\rangle$ with the smallest possible eigenvalue $E_0$. This $E_0$ determines the molecule's stability, bond length and other chemical properties of the molecules.

### The Challenge of Fermions

Electrons are **fermions**. They follow the **Pauli Exclusion Principle**: no two electrons can occupy the same quantum state. Mathematically, this means their wavefunction must be **anti-symmetric**. If you swap two electrons, the wavefunction picks up a minus sign:

$$\Psi(r_1, r_2) = -\Psi(r_2, r_1)$$

Qubits, however, are distinguishable. $|0\rangle_1 |1\rangle_2$ is not the same as $|1\rangle_1 |0\rangle_2$, and swapping them doesn't naturally generate a minus sign. To simulate electrons using qubits, we need a mathematical "translator."

## 3. Mapping Fermions to Qubits: Jordan-Wigner

We need to map the physics of electrons (orbitals being empty or occupied) to the physics of qubits (states being $|0\rangle$ or $|1\rangle$). The standard method for this is the **Jordan-Wigner Mapping**.

### Step 1: Define the Orbitals (Second Quantization)
Instead of tracking electron positions, we track **orbitals**. We assign an index $j$ to each available orbital (spatial orbital $\times$ spin).

* **Example (4 Spin-Orbitals):** Imagine a simplified model with 4 spin-orbitals, labeled $0, 1, 2, 3$.
    * Orbitals 0 & 1 could be the $1s$ shell (spin up/down).
    * Orbitals 2 & 3 could be the $2s$ shell (spin up/down).

We define fermionic operators for each orbital $j$:
* **Creation Operator ($a^\dagger_j$):** Creates an electron in orbital $j$.
* **Annihilation Operator ($a_j$):** Removes an electron from orbital $j$.
* **Number Operator ($n_j = a^\dagger_j a_j$):** Counts if an electron is there (1) or not (0).

### Step 2: Map Occupancy to Qubits
This is the easy part. We map the *presence* of an electron to the state of a qubit.
* **Qubit $j$ represents Orbital $j$.**
* **State $|0\rangle_j$:** Orbital $j$ is EMPTY.
* **State $|1\rangle_j$:** Orbital $j$ is OCCUPIED.

**Example:**
* State with electrons in orbitals 0 and 2: $|1\rangle_2 |0\rangle_1 |1\rangle_0$ (or $|101\rangle$).

### Step 3: Map the Operators (The Parity String)
This is the hard part. We must map the operators $a_j$ and $a^\dagger_j$ to Pauli matrices ($X, Y, Z$) in a way that preserves the anti-symmetry (the minus sign when swapping).

The Jordan-Wigner transformation does this by attaching a "string" of $Z$ operators to every creation/annihilation operator. This string calculates the **parity** of all qubits *below* index $j$.


**The Formula:**
$$a^\dagger_j \rightarrow \left( \bigotimes_{k=0}^{j-1} Z_k \right) \otimes \sigma^+_j$$
$$a_j \rightarrow \left( \bigotimes_{k=0}^{j-1} Z_k \right) \otimes \sigma^-_j$$

* **$\sigma^\pm_j$:** These are the qubit raising/lowering operators: $\sigma^+ = \frac{1}{2}(X-iY)$ and $\sigma^- = \frac{1}{2}(X+iY)$. They flip qubit $j$ (create/destroy the electron).
* **$Z_k$ string:** This acts on all qubits $0$ to $j-1$. Since $Z|1\rangle = -|1\rangle$, this string effectively counts the number of electrons "below" orbital $j$ and applies a minus sign if that number is odd. This "non-local" string(strings that operates on all qubits, instead on one specific quibt) enforces the anti-symmetry!

**Example (4 Orbitals):**
Let's map the creation operator for orbital 2, $a^\dagger_2$.
* Target qubit: 2 (Apply $\sigma^+_2$).
* Parity string: Apply $Z$ to all qubits below 2 (Qubits 0 and 1).
* **Result:** $a^\dagger_2 \rightarrow \sigma^+_2 \otimes Z_1 \otimes Z_0 = \frac{1}{2}(X_2 - iY_2) Z_1 Z_0$.

### Step 4: Transform the Hamiltonian
The molecular Hamiltonian is a sum of terms involving these operators. We substitute the qubit mappings into the Hamiltonian to get a sum of Pauli strings.

**Example (Hopping Term):**
Consider a term representing an electron hopping from orbital 0 to orbital 2: $H_{hop} = a^\dagger_2 a_0 + a^\dagger_0 a_2$.
1. Map $a^\dagger_2 \rightarrow \sigma^+_2 Z_1 Z_0$.
2. Map $a_0 \rightarrow \sigma^-_0$.
3. Multiply: $a^\dagger_2 a_0 = (\sigma^+_2 Z_1 Z_0)(\sigma^-_0) = \sigma^+_2 Z_1 (Z_0 \sigma^-_0)$.
   Since $Z_0 \sigma^-_0 = \sigma^-_0$ (check this!), the term becomes $\sigma^+_2 Z_1 \sigma^-_0$.
4. Convert to X/Y:
   $$\sigma^+_2 Z_1 \sigma^-_0 = \frac{1}{4} (X_2 - iY_2) Z_1 (X_0 + iY_0)$$
   $$= \frac{1}{4} (X_2 Z_1 X_0 + i X_2 Z_1 Y_0 - i Y_2 Z_1 X_0 + Y_2 Z_1 Y_0)$$

The final qubit Hamiltonian is a sum of these Pauli strings (like $X_2 Z_1 X_0$).

## 4. The VQE Ansatz

Unlike QAOA, where the circuit structure is dictated by the problem graph, in VQE we have to choose a circuit (Ansatz) that we think can represent the ground state of the molecule.

### Heuristic Ansatz (Hardware Efficient)
In the context of quantum computing, particularly for variational algorithms like VQE (Variational Quantum Eigensolver), Heuristic Ansatz refers to a trial wavefunction (quantum circuit) that is designed based on general principles of quantum gate operations and hardware constraints, rather than specific physical or chemical knowledge of the problem being solved. This ansatz ignores the chemistry. Instead, it asks: "What is an expressive circuit that is easy to run on this specific quantum computer?"

* The structure is several layers of single-qubit rotations ($R_y, R_z$) followed by entangling gates ($CNOT$) in a repeating pattern.

* It is easy to implement on near-term hardware (NISQ) which it has shallow depth that is less susceptible to noise. However, it doesn't know any chemistry. It might have to search a massive space to find the right state ( and prone to "Barren Plateaus" that we briefly discussed in Week 6).

* **Example:** `EfficientSU2` in Qiskit.

### Chemistry-Inspired Ansatz (UCCSD)
It is a type of trial wavefunction designed specifically to model the electron interactions within a molecule, incorporating physical principles like electron number conservation and spin symmetry, and the most common chemistry-inspired ansatz is Unitary Coupled Cluster Singles and Doubles (UCCSD).

* Structure: The circuit is mathematically derived to explore only states that make physical sense (preserving the number of electrons, spin, etc.). It starts from the Hartree-Fock state (a simple approximation where the electron fill the lowest energy orbitals). Then, the ansatz adds "excitations" (apply unitary operations to move electrons from occupied to empty orbitals).

* It can reach high precision which typically finds the ground state with fewer parameters compare to a generic circuits. However, it has a very deep circuit model which requires many $CNOT$ gates, making it hard to run on noisy hardware without error mitigation.

## 5. Execution: The Variational Principle

How do we know if we found the ground state? We rely on the **Variational Principle** of quantum mechanics.

For *any* normalized quantum state $|\psi(\theta)\rangle$, the expectation value of the Hamiltonian is always greater than or equal to the true ground state energy $E_0$:
$$\langle \psi(\theta) | H | \psi(\theta) \rangle \ge E_0$$

**The Algorithm** using the Qiskit library can be broken down into several steps:

1.  **Initialize:** Choose an Ansatz $U(\theta)$.
2.  **Measure:** Use the Qiskit **`Estimator`** primitive to calculate the expectation value (energy) $E(\theta) = \langle \psi(\theta) | H_{molecule} | \psi(\theta) \rangle$.
    * *Note:* Since $H_{molecule}$ is a sum of Pauli strings, the Estimator measures the appropriate basis for each string (X, Y, or Z) and sums the results weighted by coefficients $c_i$.
3.  **Optimize:** Use a classical optimizer (SPSA or COBYLA) to change $\theta$ to make $E(\theta)$ smaller.
4.  **Repeat:** Until the energy stops decreasing. The final value is our approximation of the ground state energy.

## 6. Application: The LiH Molecule

One of the application that the chemistry-inspired ansatz is used is Lithium Hydride (LiH), which is simple enough to simulate, complex enough to be interesting.

**Orbitals**: LiH has 4 electrons. We might map this to 4 spatial orbitals $\rightarrow$ 8 spin-orbitals $\rightarrow$ 8 Qubits.

We can often "freeze" core electrons that don't participate in bonding to reduce the qubit count (e.g., down to 4 or 6 qubits). Then, we can use Qiskit Nature to generate the Hamiltonian and apply the Jordan-Wigner mapper. As a result, running this VQE produces a curve of Energy vs. Interatomic Distance. The minimum of this curve tells us the bond length of the molecule.
